# tb.lx Data Science Challenge - Part I
----
----
## Introduction

Dear applicant,

Congratulations on passing the first screening! We’re excited to get to know you better and get a better feeling of your competences. In this round, we will test you on your problem-solving skills and data science experience by giving you a case to solve.

After handing us over your solution, we will review it and let you know our feedback. In the case you have passed, you will be called to an on-site interview. During the interview, you’ll get the opportunity to explain your solution and the steps that you took to get there. We've prepared this notebook for you, to help you walk us through your ideas and decisions.

If you're not able to fully solve the case, please elaborate as precisely as you can:

- Which next steps you'd be taking;
- Which problems you'd be foreseeing there and how you'd solve those.

In case you have any questions, feel free to contact ana.cunha@daimler.com or sara.gorjao@daimler.com for any more info. 

Best of luck!

## Context

Working with GPS data is part of tb.lx day to day life. We need to extract and analyze patterns from fleets in order to enable intelligence over it. Just by knowing the history of the position of vehicles (latitude, longitude, and timestamp) it is possible to answer the questions we ask below. Take into consideration the following concepts:

**Frequently stopping location** - Delimited location where vehicles stop regularly with a specific purpose. In the trucks world it can be warehouses, fuel stations, rest locations, etc;

**Trip** - This is what results of a vehicle that moved from a '*frequently stopping location A*' to a '*frequently stopping location B*'. To make it short let's say trip(A,B);

**Trajectory** - Is the actual path of the vehicle that he took to make the *trip*(A,B);

<img src="../images/concepts.png" style="width: 700px;"/>


## Data:

In this challenge, we ask you to perform simple analyses on a vehicle telematics dataset. The dataset to use in order to answer the questions can be found associated with the paper: [Vehicle Energy Dataset (VED), A Large-scale Dataset for Vehicle Energy Consumption Research](https://arxiv.org/abs/1905.02081).

#### Important: You cannot use the `Trip` column for your calculations.

## Tasks:

1. How would you find the *frequently stopping locations* from the data?
2. What is the most popular *frequently stopping location*? How many vehicles start or end their *trips* there? Please identify the *frequently stopping location* using its bounding box coordinates.
3. What is the most frequent *trip*? How many statistically different *trajectories* make up this trip?
4. What are the *trips* with the highest and lowest average speed?
5. Discuss the anonymization process used by the dataset authors. Are there any obvious flaws? Can you devise a way to counter it?


## Requirements:

- Solution implemented in Python3.6+;
- Provide requirements.txt to test the solution in the same environment;
- Write well structured, documented, maintainable code;
- Write sanity checks to test the different steps of the pipeline;

## Downloading the data

First we need to download the data for this analysis. We'll start by creating a folder in which to download the data and then download the data into this folder

**Note:** The data folders are zipped using 7zip, so in order to unzip them 7zip must be installed in the machine to use the `7z` command

In [1]:
# Create a folder in which to place the data (Remove it if already exists)
!rm -r VED_data/

!mkdir VED_data

# Download the data
!wget https://raw.githubusercontent.com/gsoh/VED/master/Data/VED_DynamicData_Part1.7z
!wget https://raw.githubusercontent.com/gsoh/VED/master/Data/VED_DynamicData_Part2.7z
!wget -P VED_data https://raw.githubusercontent.com/gsoh/VED/master/Data/VED_Static_Data_ICE\&HEV.xlsx
!wget -P VED_data https://raw.githubusercontent.com/gsoh/VED/master/Data/VED_Static_Data_PHEV\&EV.xlsx

# Unzip the data into the folder created
# NOTE: 7z needs to be installed in the machine
!7z x VED_DynamicData_Part1.7z -oVED_data
!7z x VED_DynamicData_Part2.7z -oVED_data

# Delete the zip file
!rm *.7z

rm: cannot remove 'VED_data/': No such file or directory
--2020-03-27 18:12:35--  https://raw.githubusercontent.com/gsoh/VED/master/Data/VED_DynamicData_Part1.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.132.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.132.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82723769 (79M) [application/octet-stream]
Saving to: ‘VED_DynamicData_Part1.7z’

VED_DynamicData_Par 100%[===================>]  78,89M  4,35MB/s    in 20s     

2020-03-27 18:12:55 (3,97 MB/s) - ‘VED_DynamicData_Part1.7z’ saved [82723769/82723769]

--2020-03-27 18:12:55--  https://raw.githubusercontent.com/gsoh/VED/master/Data/VED_DynamicData_Part2.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.132.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.132.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Defining spark session and context
spark = SparkSession.builder \
           .master('local[*]') \
           .appName('tblx') \
           .getOrCreate()

sc = spark.sparkContext

# Define the correct schema for the columns in order to optimize memory managment
schema = StructType([
    StructField("DayNum", FloatType(), True),
    StructField("VehId", IntegerType(), True),
    StructField("Trip", StringType(), True),
    StructField("Timestamp(ms)", IntegerType(), True),
    StructField("Latitude[deg]", FloatType(), True),
    StructField("Longitude[deg]", FloatType(), True),
    StructField("Vehicle Speed[km/h]", FloatType(), False),
    StructField("MAF[g/sec]", FloatType(), True),
    StructField("Absolute Load[%]", FloatType(), True),
    StructField("OAT[DegC]", StringType(), True),
    StructField("Fuel Rate[L/hr]", StringType(), True),
    StructField("Air Conditioning Power[kW]", StringType(), True),
    StructField("Air Conditioning Power[Watts]", StringType(), True),
    StructField("Heater Power[Watts]", StringType(), True),
    StructField("HV Battery Current[A]", StringType(), True),
    StructField("HV Battery SOC[%]", StringType(), True),
    StructField("HV Battery Voltage[V]", StringType(), True),
    StructField("Short Term Fuel Trim Bank 1[%]", StringType(), True),
    StructField("Short Term Fuel Trim Bank 2[%]", StringType(), True),
    StructField("Long Term Fuel Trim Bank 1[%]", StringType(), True),
    StructField("Long Term Fuel Trim Bank 2[%]", StringType(), True)
])

# Read all dynamic csvs into one spark dataframe
df = spark.read.option('header', 'true').csv('./VED_data/*.csv', schema=schema)

# Keeping only the needed columns
df = df.select(["DayNum", "VehId", "Timestamp(ms)", "Latitude[deg]", "Longitude[deg]", "Vehicle Speed[km/h]"])

# Replace the string "NaN" with null values
df = df.replace("NaN", None)

# Drop rows where all values are null (sanity check)
df = df.na.drop(how="all")

# Get a sample of the dataframe (using just the first two weeks here)
first_two_weeks = df.na.drop() \
    .filter((df.DayNum >= 1) & (df.DayNum < 15)) \
    .orderBy(["VehId", "DayNum", "Timestamp(ms)"])

Looking at the data the pair (DayNum, VehId) seems to define a trip

## 1- Defining stopping location

A stopping location could viewed as be a pair of (lat, long) where a trip begins (i.e. timestamp 0) or ends (i.e. maximum timestamp for that trip). This is identical to grouping by the pair of DayNum, VehId (that identifies a trip) and taking the first and last latitude and longitude of each group since rows are also sorted by their timestamp (as specified in the cell above)

However we see the anonymization process in action where the a trip starts and ends with a speed > 0, indicating that the trajectories prior to $t_{min}$ and $t_{max}$ are supressed as stated in the paper

In [3]:
grouped = first_two_weeks.groupBy(["DayNum", "VehId"])

# Expression to obtain the median function of the vehicle speed
median_function = expr('percentile_approx(`Vehicle Speed[km/h]`, 0.5)')

# Get the first and last lat, long and the avg/median speed for each pair of (DayNum, VehId)
start_locations = grouped.agg(
    first("Latitude[deg]").alias("Latitude"),
    first("Longitude[deg]").alias("Longitude"),
    mean("Vehicle Speed[km/h]").alias("Average_speed"),
    median_function.alias("Median_Speed")
)

end_locations = grouped.agg(
    last("Latitude[deg]").alias("Latitude"),
    last("Longitude[deg]").alias("Longitude"),
    mean("Vehicle Speed[km/h]").alias("Average_speed"),
    median_function.alias("Median_Speed")
)

# Join the two dataframes together so we have a list of stopping locations (either start or end) and the 
# avg/median speed for every trip
# .distinct() to remove already existing stopping locations
stopping_locations = start_locations.unionAll(end_locations).distinct()

## 2- What are the most popular stopping locations? How many vehicles start or end their trips there?

First define the bounding box coordinates by adding and subtracting 0.0025º to lat and long and rounding the result to 3 decimal places. Rounding the result will allow for points in close locations to have the same bounding box coordinates.

Here the popularity of a location will be measured by the number of distinct vehicles that start or end their trip their. It is important to consider the distinct number of vehicles that start or end their trips in this location rather than just the total number of trips, in order to make sure we are not influenced by a large number of trips that only a small number of users make (for example: only a small X percent of the users going to the supermarket everyday).

In [4]:
# Create the bounding box coordinates for each row
# Boxing the coordinates with +- 0.0025 which translates to around 200m around the original point
stopping_locations = stopping_locations \
                        .withColumn("top_box", round(col("Latitude") + 0.0025, 3)) \
                        .withColumn("bottom_box", round(col("Latitude") - 0.0025, 3)) \
                        .withColumn("left_box", round(col("Longitude") + 0.0025, 3)) \
                        .withColumn("right_box", round(col("Longitude") - 0.0025, 3))

# The most popular stopping locations here are those locations with the most visits by distinct vehicles
stopping_locations \
    .groupBy(["top_box", "bottom_box", "left_box", "right_box"]) \
    .agg(countDistinct("VehId").alias("num_unique_vehicles"), count("VehId").alias("num_stops")) \
    .sort(col("num_unique_vehicles").desc()) \
    .show(3)

+-------+----------+--------+---------+-------------------+---------+
|top_box|bottom_box|left_box|right_box|num_unique_vehicles|num_stops|
+-------+----------+--------+---------+-------------------+---------+
| 42.277|    42.272| -83.672|  -83.677|                 44|      121|
| 42.255|     42.25| -83.672|  -83.677|                 36|       54|
| 42.233|    42.228| -83.678|  -83.683|                 33|       67|
+-------+----------+--------+---------+-------------------+---------+
only showing top 3 rows



From this we can see that the most popular stopping location has had 121 trips start or end in it, with 44 distinct vehicles. Looking at the map this location seems to be around Concordia University Ann Arbor

## 3- What is the most frequent trip?

Here we want to see what is the pair of two stopping locations that make up a trip that has been made the most times by drivers. Again, it is also interesting to look beyond the number of trips and see how many distinct drivers made this trip

In [5]:
# rename lat and long column of end_locations and join the two dataframes on DayNum and VehId
# Also round coordinates to 3 decimal places
# Droping the average and median speed from end_locations since start_locations already has that same info
trips_df = start_locations \
    .join(
        end_locations \
            .drop("Average_Speed", "Median_Speed")\
            .withColumnRenamed("Latitude", "End_latitude") \
            .withColumnRenamed("Longitude", "End_longitude"),
        ["DayNum", "VehId"]
    ) \
    .withColumn("Latitude", round(col("Latitude"), 3)) \
    .withColumn("Longitude", round(col("Longitude"), 3)) \
    .withColumn("End_latitude", round(col("End_latitude"), 3)) \
    .withColumn("End_longitude", round(col("End_longitude"), 3))

# Get the trip with the largest value of trips by distinct vehicles
trips_df \
    .groupBy(["Latitude", "Longitude", "End_latitude", "End_longitude"]) \
    .agg(countDistinct("VehId").alias("num_unique_trips"), count("VehId").alias("num_trips")) \
    .orderBy(["num_trips", "num_unique_trips"], ascending=False) \
    .show(1)

+--------+---------+------------+-------------+----------------+---------+
|Latitude|Longitude|End_latitude|End_longitude|num_unique_trips|num_trips|
+--------+---------+------------+-------------+----------------+---------+
|  42.309|  -83.677|      42.303|      -83.704|               5|        5|
+--------+---------+------------+-------------+----------------+---------+
only showing top 1 row



The most frequent trip has been made 5 times by 5 different vehicles.

Looking at this trip in a map we can see this trip begins in a comercial area and ends in the east campus of the University of Michigan. The trips starts near a starbucks and a McDonald's so I'm guessing these are 5 different people who all went out to lunch and a coffee before going to college :)

## 4- What are the trips with the highest and lowest average speed?

In [6]:
result_query = trips_df \
    .select(min("Average_Speed"), max("Average_Speed"), min("Median_Speed"), max("Median_Speed")) \
    .collect()[0]

min_avg_speed, max_avg_speed = result_query["min(Average_Speed)"], result_query["max(Average_Speed)"]
min_median_speed, max_median_speed = result_query["min(Median_Speed)"], result_query["max(Median_Speed)"]

trips_df \
    .where((trips_df["Average_Speed"] == min_avg_speed) | (trips_df["Average_Speed"] == max_avg_speed)) \
    .show()

+--------+-----+--------+---------+------------------+------------+------------+-------------+
|  DayNum|VehId|Latitude|Longitude|     Average_speed|Median_Speed|End_latitude|End_longitude|
+--------+-----+--------+---------+------------------+------------+------------+-------------+
|12.91785|  285|  42.274|  -83.675|2.6194029850746268|         0.0|      42.274|      -83.674|
|8.384305|  557|  42.288|  -83.747|104.56752873563218|       117.0|      42.239|      -83.738|
+--------+-----+--------+---------+------------------+------------+------------+-------------+



Here we can see the the lowest average speed in a trip was around 2 km/h and the highest average speed in a trip was around 104 km/h. However in cases such as speed it might prove more informative to use the median instead of the average since the average can be heavily influence by outliers (such as stopping, or speeding in a highway)

In [7]:
trips_df \
    .where((trips_df["Median_Speed"] == min_median_speed) | (trips_df["Median_Speed"] == max_median_speed)) \
    .show()

+---------+-----+--------+---------+------------------+------------+------------+-------------+
|   DayNum|VehId|Latitude|Longitude|     Average_speed|Median_Speed|End_latitude|End_longitude|
+---------+-----+--------+---------+------------------+------------+------------+-------------+
|7.7071733|  269|  42.257|  -83.696|18.852601156069365|         0.0|      42.231|       -83.68|
|1.4773134|  189|  42.269|  -83.747|5.8647373387857815|         0.0|      42.269|      -83.747|
|4.3980327|  163|  42.245|  -83.703|13.918650793650794|         0.0|      42.246|      -83.681|
|11.116269|  195|  42.289|  -83.747| 8.808783165599268|         0.0|      42.291|       -83.74|
|3.7202237|  163|  42.245|   -83.68| 9.384969325153374|         0.0|      42.253|      -83.676|
|10.532848|  189|  42.269|  -83.747| 5.963488843813387|         0.0|      42.272|      -83.745|
| 4.937109|  547|  42.245|  -83.694| 17.31063829787234|         0.0|      42.246|      -83.681|
| 12.91785|  285|  42.274|  -83.675|2.61

We see that the results for the median speed don't differ that much from the average speed. Actually both the maximum and the minimum median speed coincide with the results for the average. This might be because a portion of the start and end of the trip is supressed and so they rarely start at 0, diminishing the outliers that might influence the average.

However we also get some additional information. Looking at, for example, the first row (with `VehId == 269`) we see that the average speed is 18.9 but the median is 0. These are cases of trips were the driver was stopped (`speed==0`) for the majority of the trip and was only moving for a small part of the trip. This could lead us to investigate these cases further to search for a specific pattern

## 5- Discussing the anonymization process

Reading section _III. De-Identification_ of the paper, some points can be brought of:

- Due to the fogging mechanism, short trips might either be completely fogged or don't have any useful information. In this cases it might be better to filter out short trips.

- Due to the fencing mechanism trips some stopping points outside the city that may be useful to know are also supressed, discarding this information. In the authors defence, their work is more pointed to the study of fuel and energy consumption than rather the study of trips

- Common stopping points in the middle of the trip are not removed. Both start and end locations are supressed from the trips, but stopping locations in the middle of the trip are not. Say for example the user always goes by the same drive-thru restaurant in the middle of the trip, this might lead to possibly identifying the user

- Drivers that always take the same routes might also not be completly anonymized. Granted that the start and end locantions are supressed, but if the driver always performs the same trajectory, than this can lead to possible identification of the driver. This may be addressed by removing common locations where a driver frequently stops in his trip, although  it impacts the quality of the data

- Furthermore, it may also be possible to get a representation of the driver, either by the route, the average speed, car-consumption information, etc.

- Considering the last two points with eletric or hybrid vehicles may narrow down the search of a drivers identity due to the rarity of these types of vehicles